# Assignment 1 #
Karthik Karavatt
2061996
5

In [69]:
import cv2 as cv
# Read the image
img = cv.imread("digits.png")

## Task 1 Data Preparation 

### 1.1 
Split the image into 5000 images of size 20x20 pixels each. Each image contains a digit.
Store this data in a numpy array

We know the digit is 20x20 pixels. So we can split the image into 5000 images of size 20x20 pixels each.
As the image is 1000x500 pixels, we can split the image into 50 rows and 100 columns.

In [70]:
import numpy as np

rows, cols, _ = img.shape
size = 20

num_rows = rows // size
num_cols = cols // size

data = np.empty((num_rows, num_cols, size, size, 3), dtype=img.dtype)

for i in range(num_rows):
    for j in range(num_cols):
        data[i, j] = img[i*size:(i+1)*size, j*size:(j+1)*size]

### 1.2
Split the data into training and test sets. Use the first 1000 images for training and the rest for testing.
As each row is a different digit, there are 5 rows of each digit and the digits go up to 9, we can use one row of each digit for training and the rest for testing. So we take one row from every 5 rows for training and the rest for testing
.

In [71]:
# Select every 5th row for testing
test_data = data[::5]

# Select all but every 5th row for training
train_data = np.delete(data, np.arange(0, data.shape[0], 5), axis=0)

### 1.3
Write the images to a file concurrently using threads. The images should be written to the train and test directories.

In [72]:
import os
import glob
import cv2 as cv
import concurrent.futures

# Specify the directories
directories = ['train', 'test']

for directory in directories:
    # Check if the directory exists
    if os.path.exists(directory):
        # Use glob to match the file pattern 'jpg'
        files = glob.glob(f'{directory}/*.jpg')

        # Use os.remove() to delete each file
        for file in files:
            os.remove(file)

# Create directories for the train and test images
os.makedirs('train', exist_ok=True)
os.makedirs('test', exist_ok=True)

def write_image(directory, i, j, data):
    cv.imwrite(f'{directory}/img_{i}_{j}.jpg', data[i, j])

# Write the training images to the 'train' directory
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(train_data.shape[0]):
        for j in range(train_data.shape[1]):
            executor.submit(write_image, 'train', i, j, train_data)

# Write the testing images to the 'test' directory
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(test_data.shape[0]):
        for j in range(test_data.shape[1]):
            executor.submit(write_image, 'test', i, j, test_data)
